# Determine Number of Rolls and Subdirectories on FTP

#### Updated: Dec 8, 2022

#  

Determine how many subdirectories and images there are. Estimate how long it would take the download all subdirectories. Estimate what would be the total size.

In [1]:
import pandas as pd
import ftplib

In [2]:
process_on_VDI = True

rootDir_local = 'C:/Users/rnaidoo/Documents/Projects_data/Alouette_I/'
if process_on_VDI:
    logDir = '//scientific/L-MP-Data/Massive files/Python/rnaidoo/Alouette_I/'
else:
    logDir = rootDir_local + '05_result/'

In [3]:
HOSTNAME = "donnees-data.asc-csa.gc.ca"
USERNAME = "Anonymous"
PASSWORD = ""

# Connect FTP Server
ftp = ftplib.FTP(HOSTNAME, USERNAME, PASSWORD)
print('Connected to ftp server: ' + HOSTNAME)

ftp_rootpath = '/users/OpenData_DonneesOuvertes/pub/AlouetteData/Alouette Data/'

Connected to ftp server: donnees-data.asc-csa.gc.ca


#    

#### Determine how many subdirectories and images there are:

In [4]:
df_record = pd.DataFrame()
ftp.cwd(ftp_rootpath)
rolls_list = ftp.nlst()
for roll in rolls_list:
    ftp.cwd(ftp_rootpath)
    ftp.cwd(roll)
    subdir_list = ftp.nlst()
    for subdir in subdir_list:
        ftp.cwd(ftp_rootpath)
        ftp.cwd(roll)
        ftp.cwd(subdir)
        img_list = ftp.nlst()
        df_record_ = pd.DataFrame({
            'Roll': roll,
            'Subdirectory': subdir,
            'images': len(img_list),
            'subdir_id': roll + '_' + subdir
        }, index=[0])
        df_record = pd.concat([df_record, df_record_], axis=0, ignore_index=True)

In [5]:
df_record.to_csv(logDir + 'image_inventory.csv', index=False)
df_record

,Roll,Subdirectory,images,subdir_id
0,R014207815,3488-15A,273,R014207815-3488-15A
1,R014207815,3489-15A,281,R014207815-3489-15A
2,R014207815,3490-15A,198,R014207815-3490-15A
3,R014207815,3491-8A,289,R014207815-3491-8A
4,R014207815,3492-8A,334,R014207815-3492-8A
...,...,...,...,...
2633,R014207980,994-B,375,R014207980-994-B
2634,R014207980,995-B,376,R014207980-995-B
2635,R014207980,996-A,348,R014207980-996-A
2636,R014207980,997-A,377,R014207980-997-A


In [6]:
print('There are ' + str(len(df_record)) + ' subdirectories.')

df1 = pd.DataFrame()
for roll in df_record['Roll'].unique():
    df_roll = df_record.loc[df_record['Roll'] == roll] 
    df1_ = pd.DataFrame({
        'Roll': roll,
        'n_subdirs': len(df_roll)
    }, index=[0])
    df1 = pd.concat([df1, df1_], ignore_index=True)
mean = df1['n_subdirs'].mean()
std_dev = df1['n_subdirs'].std()
print('There are on average ' + str(round(mean,0)) + ' +/- ' + str(round(std_dev,0)) + ' subdirectories in each roll.')

mean = df_record['images'].mean()
std_dev = df_record['images'].std()
print('There are on average ' + str(round(mean,0)) + ' +/- ' + str(round(std_dev,0)) + ' images in each subdirectory.')

There are 2638 subdirectories.
There are on average 59.0 +/- 6.0 subdirectories in each roll.
There are on average 336.0 +/- 55.0 images in each subdirectory.
